In [1]:
from pathlib import Path
from collections import defaultdict

import pandas as pd

In [4]:
GFF_PATH = Path("../data/raw/gencode.v19.annotation.gff3")
OUT_CSV = Path("../data/interim/gene_lengths_gencode_v19_grch37.csv")

print("Leyendo GFF3 desde:", GFF_PATH)
print("Guardado gene_lengths_bp en CSV:", OUT_CSV)

Leyendo GFF3 desde: ../data/raw/gencode.v19.annotation.gff3
Guardado gene_lengths_bp en CSV: ../data/interim/gene_lengths_gencode_v19_grch37.csv


In [3]:
# gene_id -> (transcript_id -> length_bp)
gene_tx_lengths = defaultdict(lambda: defaultdict(int))

with GFF_PATH.open() as f:
    for i, line in enumerate(f, start=1):
        # Saltar comentarios
        if line.startswith("#"):
            continue

        parts = line.rstrip("\n").split("\t")
        if len(parts) < 9:
            continue

        chrom, source, feature, start, end, score, strand, phase, attrs = parts

        # Solo exones
        if feature != "exon":
            continue

        # Parsear atributos
        attr_dict = {}
        for item in attrs.split(";"):
            item = item.strip()
            if not item or "=" not in item:
                continue
            key, value = item.split("=", 1)
            attr_dict[key] = value

        gene_id = attr_dict.get("gene_id")
        transcript_id = attr_dict.get("transcript_id")

        if gene_id is None or transcript_id is None:
            continue

        length = int(end) - int(start) + 1
        gene_tx_lengths[gene_id][transcript_id] += length
        
        if i % 500000 == 0:
            print(f"Procesadas {i:,} líneas...")

print("Número de genes encontrados:", len(gene_tx_lengths))

Procesadas 500,000 líneas...
Procesadas 1,000,000 líneas...
Procesadas 2,000,000 líneas...
Procesadas 2,500,000 líneas...
Número de genes encontrados: 57773


In [5]:
# Para cada gen, nos quedamos con la longitud de su transcrito más largo
gene_lengths = {}
for gene_id, tx_dict in gene_tx_lengths.items():
    gene_lengths[gene_id] = max(tx_dict.values())

gene_lengths_bp = pd.Series(gene_lengths, name="gene_length_bp")
print("Primeros gene_ids (con versión):")
print(gene_lengths_bp.head())

# Quitar la versión del Ensembl ID (ENSG...X.Y -> ENSG...X)
gene_lengths_bp.index = gene_lengths_bp.index.str.split(".").str[0]

print("Primeros gene_ids (sin versión):")
print(gene_lengths_bp.head())

# Nos aseguramos de tenerlo como DataFrame
gene_lengths_df = gene_lengths_bp.to_frame(name="gene_length_bp")

gene_lengths_df.to_csv(OUT_CSV)
print("Número de genes con longitud:", gene_lengths_df.shape[0])

Primeros gene_ids (con versión):
ENSG00000223972.4    1657
ENSG00000227232.4    1783
ENSG00000243485.2     712
ENSG00000237613.2    1187
ENSG00000268020.2     840
Name: gene_length_bp, dtype: int64
Primeros gene_ids (sin versión):
ENSG00000223972    1657
ENSG00000227232    1783
ENSG00000243485     712
ENSG00000237613    1187
ENSG00000268020     840
Name: gene_length_bp, dtype: int64
Número de genes con longitud: 57773
